In [1]:
import pandas as pd
import numpy as np
import string
import autokeras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_excel("Data_Train.xlsx")

In [3]:
df.shape

(10683, 11)

In [4]:
df = df.dropna(how="any")

In [5]:
df["Day"] = df["Date_of_Journey"].str.split("/").str[0]
df["Month"] = df["Date_of_Journey"].str.split("/").str[1]
df["Year"] = df["Date_of_Journey"].str.split("/").str[2]

In [6]:
df["Day"] = df["Day"].astype("int")
df["Month"] = df["Month"].astype("int")
df["Year"] = df["Year"].astype("int")

In [7]:
df = df.drop(["Date_of_Journey"], axis=1)

In [38]:
df["Arrival_Time"] = df["Arrival_Time"].str.split(" ").str[0]

In [41]:
df["Total_Stops"] = df["Total_Stops"].replace("non-stop", "0 stop")
df["Total_Stops"] = df["Total_Stops"].str.split(" ").str[0]
df["Total_Stops"] = df["Total_Stops"].astype("int")

In [44]:
df["Departure_Hour"] = df["Dep_Time"].str.split(":").str[0]
df["Departure_Minute"] = df["Dep_Time"].str.split(":").str[1]

In [47]:
df = df.drop(["Dep_Time"], axis=1)

In [49]:
df["Arrival_Hour"] = df["Arrival_Time"].str.split(":").str[0]
df["Arrival_Minute"] = df["Arrival_Time"].str.split(":").str[1]

In [50]:
df = df.drop(["Arrival_Time"], axis=1)

In [54]:
df["R1"] = df["Route"].str.split("→ ").str[0]
df["R2"] = df["Route"].str.split("→ ").str[1]
df["R3"] = df["Route"].str.split("→ ").str[2]
df["R4"] = df["Route"].str.split("→ ").str[3]

In [59]:
df = df.drop(["Route"], axis=1)

In [56]:
df["R1"] = df["R1"].fillna("None")
df["R2"] = df["R2"].fillna("None")
df["R3"] = df["R3"].fillna("None")
df["R4"] = df["R4"].fillna("None")

In [62]:
df = df.drop(["Duration"], axis=1)

In [65]:
cols = ["Airline", "Source", "Destination", "Additional_Info", "R1", "R2", "R3", "R4"]

In [66]:
def label_encoder(column):
    le = LabelEncoder().fit(column)
    print(column.name, le.classes_)
    return le.transform(column)

In [68]:
for col in cols:
    df[col] = label_encoder(df[col])

Airline ['Air Asia' 'Air India' 'GoAir' 'IndiGo' 'Jet Airways'
 'Jet Airways Business' 'Multiple carriers'
 'Multiple carriers Premium economy' 'SpiceJet' 'Trujet' 'Vistara'
 'Vistara Premium economy']
Source ['Banglore' 'Chennai' 'Delhi' 'Kolkata' 'Mumbai']
Destination ['Banglore' 'Cochin' 'Delhi' 'Hyderabad' 'Kolkata' 'New Delhi']
Additional_Info ['1 Long layover' '1 Short layover' '2 Long layover' 'Business class'
 'Change airports' 'In-flight meal not included' 'No Info'
 'No check-in baggage included' 'No info' 'Red-eye flight']
R1 ['BLR ' 'BOM ' 'CCU ' 'DEL ' 'MAA ']
R2 ['AMD ' 'ATQ ' 'BBI ' 'BDQ ' 'BHO ' 'BLR' 'BLR ' 'BOM ' 'CCU' 'CCU ' 'COK'
 'COK ' 'DED ' 'DEL' 'DEL ' 'GAU ' 'GOI ' 'GWL ' 'HBX ' 'HYD' 'HYD '
 'IDR ' 'IXA ' 'IXB ' 'IXC ' 'IXR ' 'IXU ' 'IXZ ' 'JAI ' 'JDH ' 'JLR '
 'KNU ' 'LKO ' 'MAA ' 'NAG ' 'NDC ' 'PAT ' 'PNQ ' 'RPR ' 'STV ' 'TRV '
 'UDR ' 'VGA ' 'VNS ' 'VTZ ']
R3 ['AMD ' 'BBI ' 'BHO ' 'BLR' 'BOM ' 'CCU ' 'COK' 'COK ' 'DEL' 'DEL ' 'GAU '
 'GOI ' 'HBX ' 'HYD' 'H

In [70]:
X = df.drop(["Price"], axis=1)
y = df["Price"]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4242)

In [74]:
sdr = autokeras.StructuredDataRegressor(overwrite=True, max_trials=3)

In [75]:
sdr.fit(x=X_train, y=y_train, verbose=0)

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [76]:
mae, mse = sdr.evaluate(X_test, y_test, verbose=2)

67/67 - 0s - loss: 4416805.0000 - mean_squared_error: 4416805.0000 - 97ms/epoch - 1ms/step


In [77]:
y_pred = sdr.predict(X_test)

67/67 [==============================] - 0s 419us/step


In [78]:
model = sdr.export_model()

In [79]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 16)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 16)               33        
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               2176      
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128  